In [ ]:
import time
i = 1
while True:
    time.sleep(5)
    print(i)
    i += 1



1


In [12]:
!python3 main.py --model_name distilbert  --num_epochs 200 --batch_size 32 --context_window 512 --data_size None --strategy cls


Traceback (most recent call last):
  File "/data/gpfs/projects/punim2662/LLM_routing/LLM_routing/main.py", line 1, in <module>
    from dataset.model_loader import ModelLoader
  File "/data/gpfs/projects/punim2662/LLM_routing/LLM_routing/dataset/model_loader.py", line 1, in <module>
    from transformers import AutoModelForCausalLM, AutoTokenizer
ModuleNotFoundError: No module named 'transformers'
